# RF回線計算（レーダ照射、RCS反射）

In [2]:
import numpy as np
import quaternion

import sys
sys.path.append('../')
sys.path.append('../utility')

#座標系クラスのインポート
from COORDINATION_SYSTEMS import ECEF, LLA, NED, FRD, AC_ATTITUDE, LOCAL, SENSOR_ATTITUDE, FOCAL_POSITION, FOCAL_ANGLE
#WGS84座標変換クラスのインポート
from WGS84_COORDINATE_CONVERT import WGS84_COORDINATION_CONVERT
#座標変換クラスのインポート
from ATTITUDE_CONVERT import ATTITUDE_CONVERT
#センサ取得情報クラスのインポート
from SENSOR_FOCAL import SENSOR_FOCAL
#センサ性能クラスのインポート
from SENSOR_PERFORMANCE import FOV
#センサモデルクラスのインポート
from SENSOR_MODEL import SENSOR_RECOGNITION

0.7071067811865474 3.3306690738754696e-16 -0.7071067811865474
-3.3306690738754696e-16 0.9999999999999998 5.551115123125783e-17
0.31961524227066307 -0.19999999999999987 -0.6464101615137753
0.19999999999999982 0.40000000000000013 0.6000000000000005
6.470853991243017e-16 0.707106781186547 0.7071067811865467
0.7071067811865474 3.3306690738754726e-16 -0.7071067811865474


### クラス：アンテナ利得（σi)<br>
入力：Focal_Az、Focal_El<br>
出力：σi　単位なし（無次元量）<br>
$$
 \sigma_{i} (Az,El)
$$

In [3]:
class AntGain:
    #コンストラクタ
    def __init__(self):
        self.dSumAz = 0
        self.dSumEl = 0
        self.Focal_Az = []
        self.Focal_El = []
    
    #加算（リセット処理あり）
    #入力：Focal_Az
    #入力：Focal_El
    #入力:flag<0>:前回値に加算、flag<1>：前回値クリア後加算
    def SetGain(self,focal_ang=FOCAL_ANGLE(),flag=0):
        if (flag == 1):
            self.dSumAz = 0
            self.dSumEl = 0
            self.Focal_Az = []
            self.Focal_El = []

        self.dSumAz += focal_ang.focal_az
        self.dSumEl += focal_ang.focal_el
        self.Focal_Az.append(focal_ang.focal_az)
        self.Focal_El.append(focal_ang.focal_el)
        
        print("Az",self.dSumAz,"El",self.dSumEl)
        #print("Az[]",self.Focal_Az[len(self.Focal_Az)-1],"El[]",self.Focal_El[len(self.Focal_El)-1])

    #ゲイン取得
    #出力 AntGain
    def GetGain(self):
        #暫定　結果：1
        self.dSumAz = 1
        self.dSumEl = 1
        return self

In [4]:
#使用例
AntGain = AntGain()
AntGain.SetGain(FOCAL_ANGLE(1,2))
AntGain.SetGain(FOCAL_ANGLE(10,20))

print(AntGain.GetGain())


Az 2 El 1
Az 22 El 11


### クラス：ＲＣＳ(σrcs)
入力：到来Az、到来El<br>
出力：σrcs　単位なし（無次元量）<br>
$$
 \sigma_{RCS} (Az,El)
$$

In [5]:
class Rcs:
    #コンストラクタ
    def __init__(self):
        self.dSumRcs = 0
        self.dAz = []
        self.dEl = []
    
    #加算（リセット処理あり）
    #入力：Focal_Az
    #入力：Focal_El
    #入力:flag<0>:前回値に加算、flag<1>：前回値クリア後加算
    def SetRcs(self,focal_ang=FOCAL_ANGLE(),flag=0):
        if (flag == 1):
            self.dSumRcs = 0
            self.dAz = []
            self.dEl = []

        #計算式　現状未定
        #self.dSumRcs += 
        self.dAz.append(focal_ang.focal_az)
        self.dEl.append(focal_ang.focal_el)
        
        print("Rcs",self.dSumRcs)

    #Rcs取得
    #出力
    def GetRcs(self):
        #暫定　結果：1
        self.dSumRcs = 1
        return self.dSumRcs

In [5]:
#使用例
SencerRcs = Rcs()
SencerRcs.SetRcs(FOCAL_ANGLE(1,2))

print(SencerRcs.GetRcs())

Rcs 0
1


### クラス：ＲＦ損失計算

メソッド：RF大気減衰(τ）<br>
入力：Tempreture　単位 K（ケルビン）<br>
入力：distance 単位m<br>
入力：altitude 単位m <br>
出力：1 (今は実装なし）<br>
$$
 \tau(Tempretur, distance, altitude) = 1
$$

In [37]:
import math

class RfLos():
    #コンストラクタ
    def __init__(self,dTemp=0,dDis=0,dAlt=0):
        self.dTemp = dTemp
        self.dDis = dDis
        self.dAlt = dAlt
        self.dPtxsm = 0
        self.dPref = 0
        self.dPrx = 0
    
    #取得
    def GetRfLos(self):
        return self
    
    #RF対気減衰
    #入力：Temp
    #入力：Dis
    #入力：Alt
    def CalcRf(self,dTemp=0,dDis=0,dAlt=0):
        print("Rf",1)        
        return 1

    #メソッド：伝達電力
    def CalcTrancePow(self,dPtx=0,dGain=0,dDis=0,dRfLos=0):
        self.dPtxsm = (math.tau * dGain)/(4 * math.pi * math.pow(dDis,2)) * dPtx
        print("Ptxsm",self.dPtxsm)
        return self.dPtxsm 
    
    #メソッド：受信電力
    def CalcRefPow(self,dRcs=0):
        self.dPref = dRcs * self.dPtxsm
        print("Pref",self.dPref)
        return self.dPref
    
    #メソッド：受信電力
    def CalcRcvPow(self,dGain=0,dDis=0):
        self.dPrx = (math.tau * dGain) / (4 * math.pi * math.pow(dDis,2)) * self.dPref
        
        #print("tau",math.tau,math.tau*dGain)
        #print(" ",(4 * math.pi * math.pow(dDis,2)))
        #print("Pref",self.dPref)
        print("Prx",self.dPrx)
        return self.dPrx
    
#los = RfLos(10,20,30)
#data = los.GetRfLos()
#print("Tmp",data.dTemp,"Dis",data.dDis,"Alt",data.dAlt,"Ptxsm",data.dPtxsm,"Pref",data.dPref,"Prx",data.dPrx)

In [38]:
#使用例
Los = RfLos(10,20,30)
calc = Los.CalcRf(1,2,3)
calc = Los.CalcTrancePow(1,2,3,4)
calc = Los.CalcRefPow(10)
calc = Los.CalcRcvPow(1,10)


Rf 1
Ptxsm 0.1111111111111111
Pref 1.1111111111111112
Prx 0.005555555555555556


メソッド：伝達電力<br>
入力：発送電力 Ptx　単位：Ｗ（ワット）<br>
入力：アンテナ利得 σi<br>
入力：距離 R(distanceのこと）<br>
入力：RF大気減衰τ
出力：伝達電力Ptxsm　単位：Ｗ（ワット）<br>
$$
 P_{txsm} = \frac{\tau \sigma_i}{4 \pi R^2} P_{tx}
$$

In [7]:
    def CalcTrancePow(self,dPtx=0,dGain=0,dDis=0,dRfLos=0):
        self.dPtxsm = (math.tau * dGain)/(4 * math.pi * math.pow(dDis,2)) * dPtx
        print("Ptxsm",self.dPtxsm)
        return self.dPtxsm

メソッド：反射電力 <br>
入力：伝達電力 Ptx<br>
入力：RCS σrcs<br>
出力：反射電力 Pref<br>
$$
 P_{ref} = \sigma_{RCS} P_{txsm}
$$

In [ ]:
    def CalcRefPow(self,dRcs=0):
        self.dPref = dRcs * self.dPtxsm
        return self.dPref

メソッド：受信電力 <br>
入力：反射電力 Ptx<br>
入力：アンテナ利得 σrcs<br>
出力：受信電力 Prx<br>
$$
 P_{rx} = \frac{\tau \sigma_i}{4 \pi R^2} P_{ref}
$$

In [9]:
    def CalcRcvPow(self,dDis=0):
        self.dPrx = (math.tau * dGain) / (4 * math.pi * math.pow(dDis,2)) * self.dPref
        return self.dPrx